In [ ]:
!pip install openai transformers torch sentencepiece tqdm

!git clone https://github.com/churchstudio-org/openbible.git
%cd ./openbible

!python scripts/cli/list_metadata.py https://github.com/churchstudio-org/openbible/raw/main/
!cat metadata.json

In [ ]:
from scripts.bible_utils import load_metadata, load_source_bible, save_bible, split_bible_by_book
from scripts.translator_factory import get_translator
import os, shutil
from google.colab import files

metadata = load_metadata()
generated_versions = []

for entry in metadata:
    version = entry.get("version")
    model_name = entry.get("model")
    source_version = entry.get("source")
    prompt = entry.get("prompt")

    if not version or not model_name or not source_version:
        print(f"⚠️ Unable to parse incomplete metadata: {entry}")
        continue

    version_dir = os.path.join(os.getcwd(), version)
    bible_path = os.path.join(version_dir, "bible.json")

    if os.path.exists(bible_path):
        print(f"✅ {version} is already translated.")
        continue

    print(f"\n🌐 Start {version} translation using {model_name}...")
    translator = get_translator(model_name, prompt)
    source_bible = load_source_bible(source_version)

    translated_bible = []
    for book in source_bible:
        translated_bible.append(translator.translate_book(book))

    save_bible(translated_bible, version_dir)
    split_bible_by_book(translated_bible, version_dir)
    print(f"📚 {version} salvo em {version_dir}")

    generated_versions.append(version_dir)
    translator.close()

for version_dir in generated_versions:
    zip_name = f"{os.path.basename(version_dir)}.zip"
    shutil.make_archive(os.path.splitext(zip_name)[0], 'zip', version_dir)
    files.download(f"{os.path.splitext(zip_name)[0]}.zip")
